In [1]:
# data load

from glob import glob

tweet_list = []
label_list = []

for index, file in enumerate(glob('data/3000tweet/raw_texts/**/*.txt', recursive=True)):                                                                 # labels
    label_list.append(file.split("/")[-2]),
    tweet_list.append((open(file, encoding="windows-1254").read().replace('\n', ' ').strip().lower()))      # tweets

print("3000 tweet  :  ", len(tweet_list))
print("3000 label  :  ", len(label_list))


3000 tweet  :   3000
3000 label  :   3000


In [2]:
print("toplam veri sayısı : ",len(label_list))
value1 = [i for i in label_list if i in '1']
value2 = [i for i in label_list if i in '2']
value3 = [i for i in label_list if i in '3']
print("1.sınıf : ",len(value1))
print("2.sınıf : ",len(value2))
print("3.sınıf : ",len(value3))

toplam veri sayısı :  3000
1.sınıf :  756
2.sınıf :  1287
3.sınıf :  957


In [3]:
print("3000 tweet  :  ", tweet_list)

3000 tweet  :   ['#turkcell superonline bir milyon evin kapisina fiber internet goturdu bizde niye yok bize niye getirmiyorlar :))', 'bi̇r evde olmamasi gereken 3 şey...1.turkcell 2. di̇gi̇turk 3.keri̇z anten çi̇çeği̇', '" en ucuz iphone 4s fiyatını hangi operatör sunuyor?  ohasın', '1 gb internet pakedim de bitti lanet turkcell.', 'her ay ben size 1000 tl yakin para oduyorum hizmet istiyorum', 'inceledimmm ve kiyasladimm gercktnn cokk yuksekk rakiplerine gore bi fiyatlarla almak akilkari degl sanrm', 'lanet olasın turkcell dedim', "dur şu turkcell'e mesaj atıyım da sevgilisinden geldi sansın enayi!", "gerçekten ttnetden hızlıysa esos geçecem superonline'a sinir oldum valla bu kadar yavaş olamaz birnet :s", 'oc musteri hizmetleri kukla ettiniz amk turkcell beni sessimden tanirmis', 'bre turkcell ! tam bi hırsızsın ! seni bi daha kullanani esekler kovalasin!', "kardeşim tanıdıgım bi piskiyatır var.. numarasını vereyim.. tansaş,turkcell,avea vb.'den gelen mesajları ona da atarsın ;)", 'o

In [4]:
# preprocessing

import Stemmer
import re


def splitIntoStem(message):

    
    return [removeNumeric(stripEmoji(singleCharacterRemove(removePunctuation
                                                           (removeHyperlinks
                                                            (removeHashtags
                                                             (removeUsernames
                                                              (stemWord(word)))))))) for word in message.split()]
def stemWord(tweet):

    ################################## for turkish
    #stemmer = Stemmer.Stemmer('turkish')
    #return stemmer.stemWord(tweet).lower()
    
    return tweet.lower()

#remove usernames
def removeUsernames(tweet):
    return re.sub('@[^\s]+', '', tweet)


#remove hashtag
def removeHashtags(tweet):
    return re.sub(r'#[^\s]+', '', tweet)

#remove link
def removeHyperlinks(tweet):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', tweet)

#remove numeric character
def removeNumeric(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

#remove punctuation
def removePunctuation(tweet):

    return re.sub(r'[^\w\s]','',tweet)

#remove single character
def singleCharacterRemove(tweet):
    return re.sub(r'(?:^| )\w(?:$| )', ' ', tweet)

#remove emoji
def stripEmoji(text):

    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    return RE_EMOJI.sub(r'', text)

In [5]:
tweet_list[0]

'#turkcell superonline bir milyon evin kapisina fiber internet goturdu bizde niye yok bize niye getirmiyorlar :))'

In [6]:
for i,k in enumerate(tweet_list):
    tweet_list[i] = " ".join(splitIntoStem(k)).split()

In [7]:
tweet_list[0]

['superonline',
 'bir',
 'milyon',
 'evin',
 'kapisina',
 'fiber',
 'internet',
 'goturdu',
 'bizde',
 'niye',
 'yok',
 'bize',
 'niye',
 'getirmiyorlar']

In [8]:
import string

def sumTweetWords(tweet_list):
    new_value = ""
    for i in tweet_list:
        value = " ".join(i) 
        new_value = new_value + " " +value
    print("len(tweet_list) : ",len(new_value.split()))

sumTweetWords(tweet_list)

len(tweet_list) :  32588


In [9]:
# remove stopwords

from nltk.corpus import stopwords


def removeStopWords(tweet_list):
    
    filtered_stopwords = []
    filtered_stopwords_number = []
    
    stop_words = stopwords.words('turkish')

    stop_words.append("bir")
    stop_words.append("iki")
    stop_words.append("üç")
    stop_words.append("dört")
    stop_words.append("beş")
    stop_words.append("altı")
    stop_words.append("yedi")
    stop_words.append("sekiz")
    stop_words.append("dokuz")
    stop_words.append("on")
    
    print("stop_words : ",stop_words)
    
    for i in tweet_list:
        filtered_sentence = [w for w in i if not w in stop_words]
        filtered_stopwords_number.append(filtered_sentence)                      # return list value
        filtered_stopwords.append(" ".join(filtered_sentence))                   # return string value
    
    return filtered_stopwords,filtered_stopwords_number                                       


tweet_list,filtered_stopwords_number = removeStopWords(tweet_list)

stop_words :  ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok', 'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer', 'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç', 'için', 'ile', 'ise', 'kez', 'ki', 'kim', 'mı', 'mu', 'mü', 'nasıl', 'ne', 'neden', 'nerde', 'nerede', 'nereye', 'niçin', 'niye', 'o', 'sanki', 'şey', 'siz', 'şu', 'tüm', 've', 'veya', 'ya', 'yani', 'bir', 'iki', 'üç', 'dört', 'beş', 'altı', 'yedi', 'sekiz', 'dokuz', 'on']


In [10]:
tweet_list

['superonline milyon evin kapisina fiber internet goturdu bizde yok bize getirmiyorlar',
 'evde olmamasi gereken şeyturkcell digiturk keriz anten çiçeği',
 'ucuz iphone s fiyatını hangi operatör sunuyor ohasın',
 'gb internet pakedim bitti lanet turkcell',
 'ay ben size tl yakin para oduyorum hizmet istiyorum',
 'inceledimmm kiyasladimm gercktnn cokk yuksekk rakiplerine gore bi fiyatlarla almak akilkari degl sanrm',
 'lanet olasın turkcell dedim',
 'dur turkcelle mesaj atıyım sevgilisinden geldi sansın enayi',
 'gerçekten ttnetden hızlıysa esos geçecem superonlinea sinir oldum valla kadar yavaş olamaz birnet',
 'oc musteri hizmetleri kukla ettiniz amk turkcell beni sessimden tanirmis',
 'bre turkcell tam bi hırsızsın seni bi kullanani esekler kovalasin',
 'kardeşim tanıdıgım bi piskiyatır var numarasını vereyim tansaşturkcellavea vbden gelen mesajları ona atarsın',
 'oç oldu günde bin çisi korkuttu seni',
 'turkcellde iyice uçtu yok yükle al yükle al hesaplamalarıma göre bin tl yüklese

In [11]:
filtered_stopwords_number

[['superonline',
  'milyon',
  'evin',
  'kapisina',
  'fiber',
  'internet',
  'goturdu',
  'bizde',
  'yok',
  'bize',
  'getirmiyorlar'],
 ['evde',
  'olmamasi',
  'gereken',
  'şeyturkcell',
  'digiturk',
  'keriz',
  'anten',
  'çiçeği'],
 ['ucuz', 'iphone', 's', 'fiyatını', 'hangi', 'operatör', 'sunuyor', 'ohasın'],
 ['gb', 'internet', 'pakedim', 'bitti', 'lanet', 'turkcell'],
 ['ay',
  'ben',
  'size',
  'tl',
  'yakin',
  'para',
  'oduyorum',
  'hizmet',
  'istiyorum'],
 ['inceledimmm',
  'kiyasladimm',
  'gercktnn',
  'cokk',
  'yuksekk',
  'rakiplerine',
  'gore',
  'bi',
  'fiyatlarla',
  'almak',
  'akilkari',
  'degl',
  'sanrm'],
 ['lanet', 'olasın', 'turkcell', 'dedim'],
 ['dur',
  'turkcelle',
  'mesaj',
  'atıyım',
  'sevgilisinden',
  'geldi',
  'sansın',
  'enayi'],
 ['gerçekten',
  'ttnetden',
  'hızlıysa',
  'esos',
  'geçecem',
  'superonlinea',
  'sinir',
  'oldum',
  'valla',
  'kadar',
  'yavaş',
  'olamaz',
  'birnet'],
 ['oc',
  'musteri',
  'hizmetleri',
  

In [12]:
sumTweetWords(filtered_stopwords_number)

len(tweet_list) :  28919


In [13]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

msg_train, msg_test, label_train, label_test = \
    train_test_split(tweet_list,label_list, test_size=0.2)  # random splitting

print("\n")
print("\n")
print("Message Train :", len(msg_train), "Message Test : ", len(msg_test), "Sum Data : ",
      len(msg_train) + len(msg_test))

labels = ['1','2','3']


## svm classification

svm = Pipeline([('vect', TfidfVectorizer()), ('svm', LinearSVC())])
svm.fit(msg_train, label_train)
y_pred_class = svm.predict(msg_test)

print("\n")
print("\n")
print('SVM Clasification: \n', classification_report(label_test, y_pred_class))  # classification report
print('SVM Confussion matrix: \n', confusion_matrix(label_test, y_pred_class,labels=labels))  # confusion matrix

print("\n")
print("\n")
print("SVM  Standard Accuracy : :  : \n", accuracy_score(label_test, y_pred_class))  # accuracy
print("\n")
print("\n")
print('SVM F1 score:', f1_score(label_test, y_pred_class, average='weighted'))
print("\n")
print("\n")


## Bayes classification

bayes = Pipeline([('vect', TfidfVectorizer()), ('bayes', MultinomialNB())])
bayes.fit(msg_train, label_train)
y_pred_class = bayes.predict(msg_test)

print("\n")
print("\n")
print('Bayes Clasification: \n', classification_report(label_test, y_pred_class))  # classification report
print('Bayes Confussion matrix: \n', confusion_matrix(label_test, y_pred_class,labels=labels))  # confusion matrix

print("\n")
print("\n")
print("Bayes  Standard Accuracy : :  : \n", accuracy_score(label_test, y_pred_class))  # accuracy
print("\n")
print("\n")
print('Bayes F1 score:', f1_score(label_test, y_pred_class, average='weighted'))
print("\n")
print("\n")







Message Train : 2400 Message Test :  600 Sum Data :  3000




SVM Clasification: 
              precision    recall  f1-score   support

          1       0.49      0.50      0.49       132
          2       0.66      0.69      0.67       279
          3       0.48      0.43      0.45       189

avg / total       0.56      0.57      0.56       600

SVM Confussion matrix: 
 [[ 66  29  37]
 [ 34 193  52]
 [ 36  72  81]]




SVM  Standard Accuracy : :  : 
 0.566666666667




SVM F1 score: 0.56374912905








Bayes Clasification: 
              precision    recall  f1-score   support

          1       0.59      0.30      0.39       132
          2       0.58      0.93      0.72       279
          3       0.65      0.31      0.42       189

avg / total       0.61      0.59      0.55       600

Bayes Confussion matrix: 
 [[ 39  75  18]
 [  7 259  13]
 [ 20 111  58]]




Bayes  Standard Accuracy : :  : 
 0.593333333333




Bayes F1 score: 0.550798885753






In [17]:
# sample test data predict

msg_test2=["süperonline beni çok mutlu ediyor"]
y_pred_class = svm.predict(msg_test2)
print(y_pred_class)

['1']


In [18]:
## Cross Validation


svmtfidf = Pipeline([('vect', TfidfVectorizer()), ('svm', LinearSVC())])
scorestfidf = cross_val_score(svmtfidf, tweet_list, label_list, cv=10)  # cross validation k=10

print("SVM Cross Validation (Tf-idf): \n", scorestfidf)
print("SVM Accuracy (Tf-idf): %0.2f (+/- %0.2f)" % (scorestfidf.mean(), scorestfidf.std() * 2), "\n\n")

svmtf = Pipeline([('vect', CountVectorizer()), ('svm', LinearSVC())])
scorestf = cross_val_score(svmtf, tweet_list, label_list, cv=10)  # cross validation k=10

print("SVM Cross Validation (Tf): \n", scorestf)
print("SVM Accuracy (Tf): %0.2f (+/- %0.2f)" % (scorestf.mean(), scorestf.std() * 2), "\n\n")

bayestfidf = Pipeline([('vect', TfidfVectorizer()), ('bayes', MultinomialNB())])
scorestfidf = cross_val_score(bayestfidf, tweet_list, label_list, cv=10)

print("Bayes Cross Validation (Tf-idf): \n", scorestfidf)
print("Bayes Accuracy (Tf-idf): %0.2f (+/- %0.2f)" % (scorestfidf.mean(), scorestfidf.std() * 2), "\n\n")

bayestf = Pipeline([('vect', CountVectorizer()), ('bayes', MultinomialNB())])
scorestf = cross_val_score(bayestf, tweet_list, label_list, cv=10)

print("Bayes Cross Validation (Tf) : \n", scorestf)
print("Bayes Accuracy (Tf): %0.2f (+/- %0.2f)" % (scorestf.mean(), scorestf.std() * 2))


SVM Cross Validation (Tf-idf): 
 [ 0.5448505   0.63122924  0.56810631  0.5282392   0.58803987  0.56810631
  0.52666667  0.54697987  0.56711409  0.5704698 ]
SVM Accuracy (Tf-idf): 0.56 (+/- 0.06) 


SVM Cross Validation (Tf): 
 [ 0.55813953  0.58803987  0.56810631  0.52159468  0.56146179  0.58471761
  0.50333333  0.52013423  0.54362416  0.53691275]
SVM Accuracy (Tf): 0.55 (+/- 0.05) 


Bayes Cross Validation (Tf-idf): 
 [ 0.56478405  0.54152824  0.56146179  0.52159468  0.55481728  0.5448505
  0.54333333  0.54026846  0.54362416  0.56375839]
Bayes Accuracy (Tf-idf): 0.55 (+/- 0.03) 


Bayes Cross Validation (Tf) : 
 [ 0.56810631  0.55813953  0.58471761  0.55149502  0.55481728  0.57142857
  0.52333333  0.55704698  0.57718121  0.58724832]
Bayes Accuracy (Tf): 0.56 (+/- 0.04)
